## Compute K-corrected Absolute Magnitude
Again, we assume the two bandpasses in the rest and observed-frame are the same.

In [1]:
# import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys

# use mpl style on github
mpl.rc_file('https://raw.githubusercontent.com/ywx649999311/configs/master/viz/yuBasic.rc')

### 1. Background
#### 1.1 Overview

$$m = M + DM + K(z)$$

where $m$ is the apparent magnitude, $M$ is the absolute magnitude, $DM$ is the distance modulus, and $K(z)$ is the so-called 'K-correction'. This leads to a formula for $M$,

$$M = m - DM - K(z)$$

From the notes in [K-correction](./k_correction.ipynb) notebook, we can get a final equation for the k-correction absolute magnitude as (for a power-law SED in the form of $f_{\nu}=\nu^{\alpha}$),

$$M = m - 5\log_{10}[\frac{D_{L}}{10 pc}] + 2.5 \log _{10}\left[[1+z]^{(1+\alpha)}\right]$$

#### 1.2 Approximation for the Luminosity Distance
Surely, we can integrate the equation of $D_L$ over redshift, but an approximation exists which can also take advantage of. The full formula is given by Equation 1 in [[Pen+98]](https://iopscience.iop.org/article/10.1086/313167/fulltext/37400.text.html). This approximation has an error of 0.4%. Note that $\Omega_o$ in [Pen+98] is in fact the presureless matter density---$\Omega_M$

### 2. The Code
- Function for the $D_L$ term
- Function for the k-correction term

#### 2.1 Pen's D_L approx & K_corr

In [116]:
def s_cubed(omega_m):
    """Compute the s function in Eqn 1 of Pen+98."""
    return ((1-omega_m)/omega_m)

def eta(a, omega_m):
    """Compute the eta function in Eqn 1 of Pen+98."""
    s = s_cubed(omega_m)**(1/3)
    fac1 = 2*np.sqrt(s**3 + 1)
    fac2 = (1/a**4 - 0.154*s/a**3 + 0.4304*s**2/a**2 + 0.19097*s**3/a 
            + 0.066941*s**4)**(-1/8)
    return fac1*fac2

def distMod(z, omega_m, h):
    """Compute distance module using the Pen+98 approx."""
    DM = -40 - 5*np.log10(
        (3/h)*(1+z)*(eta(1, omega_m) - eta(1/(1+z), omega_m))
    )
    return DM

In [118]:
def k_corr(z, a):
    """Compute the K-correction term given the power-law index and redshift."""
    return 2.5*(1+a)*np.log10(1+z)

#### 2.2 All together

In [124]:
def absMag(m, z, h=0.7, omega_m=0.3, alpha=-0.5):
    """Compute absolute mag k-corrected to z=0."""
    DM = distMod(z, omega_m, h)
    k_z = k_corr(z, alpha)
    return m + DM + k_z
def absMag_2(m, z, h=0.7, omega_m=0.3, alpha=-0.5):
    """Compute absolute mag k-corrected to z=2 following [Richards+06]"""
    return abs_mag(m, z, h, omega_m, alpha) - 2.5*(1+alpha)*np.log10(1+2)

##### Test

In [114]:
z = 1
h = 0.7
alpha = -0.5
m = 19.1
omega_m = 0.3

In [125]:
print(f"M(z=0): {absMag(m, z)}")
print(f"M(z=0): {absMag_2(m, z)}")

M(z=0): -24.627148589323404
M(z=0): -25.223550157722983
